In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import requests as req
import tempfile as tmpf
import numpy as np

In [61]:
%matplotlib widget

In [5]:
uk_url = 'https://api.coronavirus.data.gov.uk/v1/data?filters=areaName=United%2520Kingdom;areaType=overview&structure=%7B%22areaType%22:%22areaType%22,%22areaName%22:%22areaName%22,%22areaCode%22:%22areaCode%22,%22date%22:%22date%22,%22newCasesByPublishDate%22:%22newCasesByPublishDate%22,%22cumCasesByPublishDate%22:%22cumCasesByPublishDate%22%7D&format=csv'


In [38]:
with tmpf.NamedTemporaryFile(mode='w') as f:
    response = req.get(uk_url)
    f.write(response.text)
    data = pd.read_csv(f.name, parse_dates=['date'])

data = data.loc[:, ['date',  'newCasesByPublishDate']].sort_values('date', ascending=True)
data['daily_cases_mean'] = data.newCasesByPublishDate.rolling(7).mean()
data['increase'] = data.newCasesByPublishDate.pct_change()
data = data.set_index('date').sort_index().loc['2020-03-05':]


In [148]:
plt.style.use('seaborn')
fig, ax = plt.subplots(figsize=(9, 6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False 
fig.canvas.footer_visible = False
fig.canvas.resizable = True
fig.canvas.capture_scroll = True
fig.canvas.toolbar_visible = True
data.daily_cases_mean.pct_change().rolling(7).mean().plot(ax=ax, label='Daily percentage increase (7 days avg)');
plt.axhline(0, color='red', label='Linear growth');
plt.axhline(0.1, color='red', linestyle=':', label='7 days doubling rate');
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.set_title('Daily new cases')
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False 
fig.canvas.footer_visible = False
fig.canvas.resizable = True
fig.canvas.capture_scroll = True
fig.canvas.toolbar_visible = True
ax.bar(x=data.index, height='newCasesByPublishDate', data=data)
ax.plot('daily_cases_mean', data=data, c='red', label='7 day rolling average');
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
data.tail(14)

,newCasesByPublishDate,daily_cases_mean,increase
date,,,
2020-09-03,1735,1434.714286,0.150531
2020-09-04,1940,1529.571429,0.118156
2020-09-05,1813,1630.285714,-0.065464
2020-09-06,2988,1812.142857,0.648097
2020-09-07,2948,2032.428571,-0.013387
2020-09-08,2460,2198.857143,-0.165536
2020-09-09,2659,2363.285714,0.080894
2020-09-10,2919,2532.428571,0.097781
2020-09-11,3539,2760.857143,0.212402
